In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup

import json
import time
import copy
from time import sleep
from selenium import webdriver

In [40]:
url = 'https://www.ebay.com/urw/Google-Home-Mini-Smart-Speaker-with-Google-Assistant-Chalk-GA00210-US-/product-reviews/239112716'
url1 = 'https://www.ebay.com/urw/Microsoft-Xbox-One-Day-One-Edition-500GB-Black-Console/product-reviews/175299667'
url2 = 'https://www.ebay.com/urw/Sony-PlayStation-4-500GB-Jet-Black-Console/product-reviews/167317351'
url3 = 'https://www.ebay.com/urw/Nintendo-Switch-HAC-001-01-32GB-Console-with-Gray-Joy-Con/product-reviews/25034355190?condition=new'

url4 = 'https://www.ebay.com/urw/Milwaukee-2407-20-12V-Cordless-Drill-Driver/product-reviews/28030363313'
url5 = 'https://www.ebay.com/urw/Makita-XPH12Z-18V-Lithium-Ion-Cordless-Hammer-Drill/product-reviews/809538545'
url6 = 'https://www.ebay.com/urw/DeWalt-DCD791B-20V-Max-XR-Li-Ion-Brushless-Compact-Drill-Tool-Only-/product-reviews/13031982751'

url7 = 'https://www.ebay.com/urw/Canon-EOS-5D-Mark-III-22-3MP-Digital-SLR-Camera-Black-Body-Only-/product-reviews/113297147'
url8 = 'https://www.ebay.com/urw/Nikon-D7500-20-9MP-Digital-SLR-Camera-Black-Body-Only-/product-reviews/237096362'
url9 = 'https://www.ebay.com/urw/Nikon-D750-24-3-MP-Digital-SLR-Camera-Black-Body-Only-/product-reviews/203954517'

r = requests.get(url1)
r.status_code

200

In [ ]:
#soup.find_all('p','review-item-content rvw-wrap-spaces')[0].find('span','show-full-review').text
#soup.find_all('div','review-section-header')
#lst = [i.text for i in soup.find_all('div','reviews-divider')]

def parse_product_category(url1):
    '''
    Returns the product category for a given product
    Input: URL of the product page to be parsed
    Output: String with the product category name
    '''
    product_category = 'Error'
    r1 = requests.get(url1)
    if r1.status_code != 200:
        raise Exception("Error Unable to read HTTP page")
        
    soup1 = BeautifulSoup(r1.text,'html.parser')
    for i in range(len(soup1.find_all('a','thrd'))):
        if type(soup1.find_all('a','thrd')[i].find('span')) is not type(None):
            return soup1.find_all('a','thrd')[i].find('span').text
    return product_category
  
def parse_product_type(url1):    
    '''
    Returns the product type for a given product
    Input: URL of the product page to be parsed
    Output: String with the product type name
    '''
    product_type = 'Error'
    r2 = requests.get(url1)
    if r2.status_code != 200:
        raise Exception("Error Unable to read HTTP page")
        
    soup2 = BeautifulSoup(r2.text,'html.parser')
    
    for i in range(len(soup2.find_all('a','scnd'))):
        if type(soup2.find_all('a','scnd')[i].find('span')) is not type(None):
            return soup2.find_all('a','scnd')[i].find('span').text
    return product_type

def parse_review_author(url1,index):    
    '''
    Returns the product type for a given product
    Input: URL of the product page to be parsed
    Output: String with the product type name
    '''
    author_name = 'Error'
    r2 = requests.get(url1)
    if r2.status_code != 200:
        raise Exception("Error Unable to read HTTP page")
        
    soup2 = BeautifulSoup(r2.text,'html.parser')
    
    author_name = soup2.find_all('a','review-item-author')[index]['title']
    return author_name

def parse_review_date(url1,index):    
    '''
    Returns the product type for a given product
    Input: URL of the product page to be parsed
    Output: String with the product type name
    '''
    review_date = 'Error'
    r2 = requests.get(url1)
    if r2.status_code != 200:
        raise Exception("Error Unable to read HTTP page")
        
    soup2 = BeautifulSoup(r2.text,'html.parser')
    
    review_date = soup2.find_all('span','review-item-date')[index]['content']
    return review_date

def parse_review_text(url1,index):    
    '''
    Returns the text description for a given product review
    Input: URL of the product page to be parsed, index for the text review
    Output: String with the product review text
    '''
    review_text = 'Error'
    r2 = requests.get(url1)
    if r2.status_code != 200:
        raise Exception("Error Unable to read HTTP page")
        
    soup2 = BeautifulSoup(r2.text,'html.parser')
    if type(soup2.find_all('p','review-item-content rvw-wrap-spaces')[index].find('span','show-full-review')) is not type(None):
        review_text = soup2.find_all('p','review-item-content rvw-wrap-spaces')[index].find('span','show-full-review').text
    elif type(soup2.find_all('p','review-item-content rvw-wrap-spaces')[index].text) is not type(None):
        review_text = soup2.find_all('p','review-item-content rvw-wrap-spaces')[index].text    
    
    return review_text

def parse_nextpage_link(url1):    
    '''
    Returns the product type for a given product
    Input: URL of the product page to be parsed
    Output: String with the product type name
    '''
    nextpage_link = 'Error'
    r2 = requests.get(url1)
    if r2.status_code != 200:
        raise Exception("Error Unable to read HTTP page")
        
    soup2 = BeautifulSoup(r2.text,'html.parser')
    if len(soup2.find_all('a','spf-link'))-1 < 0:
        return None
    
    nextpage_link = soup2.find_all('a','spf-link')[len(soup2.find_all('a','spf-link'))-1]['href']
    return nextpage_link

def text_word_count(txt):    
    '''
    Returns the word count for a given test
    Input: URL of the product page to be parsed
    Output: String with the product type name
    '''
    word_count = len([elem.strip("(),.!?;:[]{}\|*%$#@/`~") 
                      for elem in txt.split(' ') 
                      if elem.strip("(),.!?;:[]{}\|*%$#@/`~")!=''])
    return word_count    

#url10 = 'https://www.ebay.com/itm/Microsoft-Xbox-One-Day-One-Edition-500GB-Black-Console/154127032955?epid=175299667&hash=item23e2afe27b:g:bjYAAOSwHXxfflK~'
#print(parse_product_category(url10))


#Store all reviews on a given page
def parse_review_page(review_url, product_url, df):
    '''
    Inserts rows into Mongo DB and pandas df 
    Input: URL of the review page and product page to be parsed, pandas Dataframe df
    Output: Dataframe df
    '''
    r = requests.get(review_url)
    if r.status_code != 200:
        raise Exception("Error Unable to read HTTP page")
    
    soup = BeautifulSoup(r.text,'html.parser')
    
    for i in range(len(soup.find_all('p','review-item-content rvw-wrap-spaces'))):
        review_text = parse_review_text(review_url,i)
        word_count = text_word_count(text)
        star_rating = float(soup.find_all('div','ebay-star-rating')[i+1].find('span','star-rating')["aria-label"].split(' ')[0])
        avg_rating = float(soup.find_all('div','ebay-star-rating')[0].find('span','star-rating')["aria-label"].split(' ')[0])
        product_name = soup.find('div','row').find('a','page-title')['title']
        product_type = parse_product_type(product_url)
        category = parse_product_category(product_url)
        author_name = parse_review_author(review_url,i)
        review_date = parse_review_date(review_url,i)
        new_row = {'category':category,'product_type':product_type,'product_name':product_name,'avg_rating':avg_rating,
                   'review_text':review_text,'word_count':word_count,'star_rating':star_rating,'author_name':author_name,
                   'review_date':review_date}
        df = df.append(new_row,ignore_index=True)
        #AA: Insert into MongoDB I
    return df
'''    
for i in range(len(soup.find_all('p','review-item-content rvw-wrap-spaces'))):
    text = soup.find_all('p','review-item-content rvw-wrap-spaces')[i].find('span','show-full-review').text
    review_text.append(text)
    word_count.append(text_word_count(text))
    star_rating.append(float(soup.find_all('div','ebay-star-rating')[i+1].find('span','star-rating')["aria-label"].split(' ')[0]))
    avg_rating.append(float(soup.find_all('div','ebay-star-rating')[0].find('span','star-rating')["aria-label"].split(' ')[0]))
    product_name.append(soup.find('div','row').find('a','page-title')['title'])
    product_type.append('gaming console')
    category.append('gaming')
    price.append(float('500'))
    
'''
#Create an empty DataFrame with all 
def create_empty_dataframe():
    category = []
    product_type = []
    product_name = []
    avg_rating = []
    review_text = []
    star_rating = []
    word_count = []
    author_name = []
    review_date = []
    df = pd.DataFrame({'category':category,'product_type':product_type,'product_name':product_name,'avg_rating':avg_rating,
                       'review_text':review_text,'word_count':word_count,'star_rating':star_rating,'author_name':author_name,
                       'review_date':review_date})    
    return df


product_url = 'https://www.ebay.com/itm/Microsoft-Xbox-One-Day-One-Edition-500GB-Black-Console/154127032955?epid=175299667&hash=item23e2afe27b:g:bjYAAOSwHXxfflK~'
review_url = url1
df = create_empty_dataframe()

def parse_all_review_pages(review_url,product_url,df):
    df = parse_review_page(review_url, product_url, df)
    next_url = parse_nextpage_link(review_url)
    while True:
        print(next_url)
        if next_url == None:
            return df
        else:
            df = parse_review_page(next_url, product_url, df)
        next_url = parse_nextpage_link(next_url)
        
    return df
'''
review_url = 'https://www.ebay.com/urw/Microsoft-Xbox-One-Day-One-Edition-500GB-Black-Console/product-reviews/175299667?pgn=2'
print(parse_review_text(review_url,0))
'''

df = parse_all_review_pages(review_url,product_url,df)
df.head(30)

'''
df = parse_review_page(review_url, product_url, df)
df.head(30)
'''

https://www.ebay.com/urw/Microsoft-Xbox-One-Day-One-Edition-500GB-Black-Console/product-reviews/175299667?pgn=2
https://www.ebay.com/urw/Microsoft-Xbox-One-Day-One-Edition-500GB-Black-Console/product-reviews/175299667?pgn=3
https://www.ebay.com/urw/Microsoft-Xbox-One-Day-One-Edition-500GB-Black-Console/product-reviews/175299667?pgn=4
https://www.ebay.com/urw/Microsoft-Xbox-One-Day-One-Edition-500GB-Black-Console/product-reviews/175299667?pgn=5
https://www.ebay.com/urw/Microsoft-Xbox-One-Day-One-Edition-500GB-Black-Console/product-reviews/175299667?pgn=6
https://www.ebay.com/urw/Microsoft-Xbox-One-Day-One-Edition-500GB-Black-Console/product-reviews/175299667?pgn=7
https://www.ebay.com/urw/Microsoft-Xbox-One-Day-One-Edition-500GB-Black-Console/product-reviews/175299667?pgn=8
https://www.ebay.com/urw/Microsoft-Xbox-One-Day-One-Edition-500GB-Black-Console/product-reviews/175299667?pgn=9
https://www.ebay.com/urw/Microsoft-Xbox-One-Day-One-Edition-500GB-Black-Console/product-reviews/17529966

In [303]:
df.shape

(11, 7)